In [76]:
sc

<SparkContext master=local[4] appName=PySpark Session>

In [77]:
spark

In [78]:
#1. create a new spark session instance

In [79]:
sc.stop()
from pyspark import SparkConf, SparkContext
config = SparkConf().setMaster("local[4]").setAppName("PySpark Session")
sc = SparkContext(conf=config)

In [80]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLSession").getOrCreate()

In [81]:
sc

<SparkContext master=local[4] appName=PySpark Session>

In [82]:
spark

In [83]:
!hdfs dfs -mkdir /flights

mkdir: `/flights': File exists


In [84]:
!hdfs dfs -put /home/hadoop/Downloads/raw_flight_data.csv /flights

put: `/flights/raw_flight_data.csv': File exists


In [85]:
flights_df = spark.read.csv('/flights/raw_flight_data.csv', 
                            header=True,
                            inferSchema=True)

In [86]:
flights_df.head(5)

[Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=11433, DestAirportID=13303, DepDelay=-3, ArrDelay=1),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=14869, DestAirportID=12478, DepDelay=0, ArrDelay=-8),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=14057, DestAirportID=14869, DepDelay=-4, ArrDelay=-15),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=15016, DestAirportID=11433, DepDelay=28, ArrDelay=24),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=11193, DestAirportID=12892, DepDelay=-6, ArrDelay=-11)]

In [87]:
flights_df.show(5) # show data in tabular format

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows



**select**
- used to select specific column from the dataframe


In [88]:
flights_df.select('DayofMonth', 'Carrier', 'OriginAirportID', 'DepDelay').show(5)

+----------+-------+---------------+--------+
|DayofMonth|Carrier|OriginAirportID|DepDelay|
+----------+-------+---------------+--------+
|        19|     DL|          11433|      -3|
|        19|     DL|          14869|       0|
|        19|     DL|          14057|      -4|
|        19|     DL|          15016|      28|
|        19|     DL|          11193|      -6|
+----------+-------+---------------+--------+
only showing top 5 rows



In [89]:
flights_df.select(['Carrier']).distinct().show()

+-------+
|Carrier|
+-------+
|     UA|
|     AA|
|     EV|
|     B6|
|     DL|
|     OO|
|     F9|
|     YV|
|     US|
|     MQ|
|     HA|
|     AS|
|     FL|
|     VX|
|     WN|
|     9E|
+-------+



**columns**
- show list of all dataframe columns

In [90]:
flights_df.columns

['DayofMonth',
 'DayOfWeek',
 'Carrier',
 'OriginAirportID',
 'DestAirportID',
 'DepDelay',
 'ArrDelay']

2. Count:
- aggregate method to show total number of values

In [91]:
# print total number of rows
flights_df.count()

2719418

In [92]:
# total number of disticts carriers
flights_df.select(['Carrier']).distinct().count()

16

3. printSchema()
- method prints names and schema datatypes

In [93]:
flights_df.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



4. where()
- It is like filter method of resilient distributed dataset with boolean conditions and statements

In [94]:
flights_df.where(flights_df.DepDelay > 0).show(5)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows



In [95]:
flights_df.where((flights_df.DepDelay > 0) & (flights_df.ArrDelay > 0)).show(5)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows



5. filter()
- same as where

In [96]:
flights_df.filter((flights_df.DepDelay > 0) & (flights_df.ArrDelay > 0)).show(5)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows



6. isin()
- filter values from dataframe by matching pattern

In [97]:
flights_df.Carrier.isin('DL', 'UA', '9E', 'F9')

Column<b'(Carrier IN (DL, UA, 9E, F9))'>

In [98]:
flights_df.where(flights_df.Carrier.isin('DL', 'UA', '9E', 'F9')).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

- Read airtposts.csv file as spark dataframe

In [99]:
airports_df = spark.read.csv('file:///home/hadoop/Downloads/airports.csv', header = True, inferSchema=True)

In [100]:
airports_df.show(5)

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
|     10754|     Barrow|   AK|Wiley Post/Will R...|
|     10551|     Bethel|   AK|      Bethel Airport|
+----------+-----------+-----+--------------------+
only showing top 5 rows



7. Join()
- join  two or more dataframes using condition

In [101]:
flights_airport_df = flights_df.join(airports_df, flights_df.OriginAirportID == airports_df.airport_id)

In [102]:
flights_airport_df.show(5)

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|       Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City|   UT|Salt Lake City In...|
|        19|        5|     DL|          14057|        14869|      -4|     -15|     14057|      Portland|   OR|Portland Internat...|
|        19|        5|     DL|          15016|        11433|      28|      24|     15016|     St. Louis|   MO|Lambert-St. Louis...|
|        19|        5|     DL|          11193|        12892|      -6|     -1

In [103]:
flights_airport_df = flights_df.join(airports_df.select(['airport_id', 'name']), flights_df.OriginAirportID == airports_df.airport_id)

In [104]:
flights_airport_df.show(5)

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City In...|
|        19|        5|     DL|          14057|        14869|      -4|     -15|     14057|Portland Internat...|
|        19|        5|     DL|          15016|        11433|      28|      24|     15016|Lambert-St. Louis...|
|        19|        5|     DL|          11193|        12892|      -6|     -11|     11193|Cincinnati/Northe...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------------+
o

8. Drop duplicates
- It will drop duplicate records from existing DataFrame


In [105]:
flights_df1 = flights_df.dropDuplicates()

#### Task 1. Calculate percentage data lost after drop duplicates

In [106]:
before = flights_df.count()
after = flights_df1.count()

In [107]:
(before - after) / before * 100

0.8249927006440348

11. Describe
- used to show summary statistics for numeric dataframe columns.

In [108]:
flights_df1.describe()

DataFrame[summary: string, DayofMonth: string, DayOfWeek: string, Carrier: string, OriginAirportID: string, DestAirportID: string, DepDelay: string, ArrDelay: string]

In [109]:
flights_df1.describe().show()

+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+
|summary|        DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|          DepDelay|          ArrDelay|
+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+
|  count|           2696983|           2696983|2696983|           2696983|           2696983|           2674774|           2673185|
|   mean|15.798996508320593| 3.900369412784582|   null|12742.459424846207| 12742.85937657004|10.618575625454712|6.7272897311633875|
| stddev| 8.801267199135454|1.9864582421701973|   null|1502.0359941370625|1501.9939589817989|36.198308432512704| 38.75007476808384|
|    min|                 1|                 1|     9E|             10140|             10140|               -63|               -94|
|    max|                31|                 7|     YV|             15376|  

12. summary()
- used for detailed summary of columns.
- summary includes, sum, count etc. and quartiles.

In [110]:
flights_df1.summary()

DataFrame[summary: string, DayofMonth: string, DayOfWeek: string, Carrier: string, OriginAirportID: string, DestAirportID: string, DepDelay: string, ArrDelay: string]

In [111]:
flights_df1.summary().show()

+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+
|summary|        DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|          DepDelay|          ArrDelay|
+-------+------------------+------------------+-------+------------------+------------------+------------------+------------------+
|  count|           2696983|           2696983|2696983|           2696983|           2696983|           2674774|           2673185|
|   mean|15.798996508320593| 3.900369412784582|   null|12742.459424846207| 12742.85937657004|10.618575625454712|6.7272897311633875|
| stddev| 8.801267199135454|1.9864582421701973|   null|1502.0359941370625|1501.9939589817989|36.198308432512704| 38.75007476808384|
|    min|                 1|                 1|     9E|             10140|             10140|               -63|               -94|
|    25%|                 8|                 2|   null|             11292|  

#### 13. Categorical Columns

In [114]:
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import *

In [117]:
# select categorical columns
categorical_data = [field.name for field in flights_df1.schema.fields if isinstance(field.dataType, StringType)]

In [116]:
categorical_data

['Carrier']

#### col(), groupBy()
- col function is used to refers a column in the dataframe
- groupBy() method to groups rows of DataFrame Based on values of one or more rows

#### 14. Get frequency values for each categorical column

In [121]:
# col function is used to refer to a column in dataframe
# where to use it - in column operations
from pyspark.sql.functions import col

In [123]:
flights_df1.filter(col('ArrDelay') > 10).count()

664460

In [125]:
for column in categorical_data:
    flights_df1.groupBy(column).count().show()

+-------+------+
|Carrier| count|
+-------+------+
|     UA|286010|
|     AA|288910|
|     EV|157218|
|     B6|121875|
|     DL|381601|
|     OO|159639|
|     F9| 35736|
|     YV| 52740|
|     US|232955|
|     MQ|112113|
|     HA| 17424|
|     AS| 68544|
|     FL| 92674|
|     VX| 34726|
|     WN|575090|
|     9E| 79728|
+-------+------+



#### 15. isnull()
- returns boolean outcome for missing values.
- true: missing value, false: no missing value

In [128]:
# check null in specific columns
flights_df1.filter(col('ArrDelay').isNull()).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        17|        3|     DL|          14869|        14771|    null|    null|
|        11|        4|     EV|          12266|        13871|    null|    null|
|        19|        5|     EV|          11618|        11433|    null|    null|
|        10|        3|     EV|          13930|        13851|    null|    null|
|         9|        2|     EV|          11292|        14107|    null|    null|
|        18|        4|     AA|          13303|        12892|    null|    null|
|        11|        4|     AA|          13930|        11292|    null|    null|
|        18|        4|     AA|          11298|        14107|    null|    null|
|        16|        2|     AA|          11278|        13930|    null|    null|
|        17|        3|     AA|          13930|      

In [130]:
# show number of missing values for one column
flights_df1.filter(col('ArrDelay').isNull()).count()

23798

In [135]:
# isNull() returns a boolean value
flights_df.select(col('ArrDelay').isNull()).show()

+------------------+
|(ArrDelay IS NULL)|
+------------------+
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
+------------------+
only showing top 20 rows



In [143]:
[count(when(isnull(column), column)).alias(column) for column in flights_df.columns]

[Column<b'count(CASE WHEN (DayofMonth IS NULL) THEN DayofMonth END) AS `DayofMonth`'>,
 Column<b'count(CASE WHEN (DayOfWeek IS NULL) THEN DayOfWeek END) AS `DayOfWeek`'>,
 Column<b'count(CASE WHEN (Carrier IS NULL) THEN Carrier END) AS `Carrier`'>,
 Column<b'count(CASE WHEN (OriginAirportID IS NULL) THEN OriginAirportID END) AS `OriginAirportID`'>,
 Column<b'count(CASE WHEN (DestAirportID IS NULL) THEN DestAirportID END) AS `DestAirportID`'>,
 Column<b'count(CASE WHEN (DepDelay IS NULL) THEN DepDelay END) AS `DepDelay`'>,
 Column<b'count(CASE WHEN (ArrDelay IS NULL) THEN ArrDelay END) AS `ArrDelay`'>]

In [142]:
flights_df.select([count(when(isnull(column), column)).alias(column) for column in flights_df.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|   27444|   29033|
+----------+---------+-------+---------------+-------------+--------+--------+



In [190]:
# create the same using col function and isNull
{column: flights_df.filter(col(column).isNull()).count() for column in flights_df.columns}

{'DayofMonth': 0,
 'DayOfWeek': 0,
 'Carrier': 0,
 'OriginAirportID': 0,
 'DestAirportID': 0,
 'DepDelay': 27444,
 'ArrDelay': 29033}

In [191]:
# create the same using col function and isNull
flights_df.select([sum(col(column).isNull().cast('int')).alias(column) for column in flights_df.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|   27444|   29033|
+----------+---------+-------+---------------+-------------+--------+--------+



#### 16. fillna()
- to replace or fill missing values of column with central tendency.
- central tendencies: [mean, median, mode]

In [194]:
# replace missing values with zeros
flights_df2 = flights_df.fillna({'DepDelay': 0, 'ArrDelay': 0})

In [196]:
# check for missing values
flights_df2.select([sum(col(column).isNull().cast('int')).alias(column) for column in flights_df2.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|       0|       0|
+----------+---------+-------+---------------+-------------+--------+--------+



#### 17. dropna()
- method is used for removing rows with null values

In [201]:
flights_df.dropna()

DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, ArrDelay: int]

In [203]:
## deleting rows with missing data:

print((flights_df.count() - flights_df.dropna().count()) / flights_df.count() * 100)

1.067618144764799


#### 18. Statistical analysis
- mean(), median(), stddiv(), quartile()

In [208]:
# mean
flights_df1.select(round(mean(col('DepDelay'))).alias('DepDelay mean')).show()

+-------------+
|DepDelay mean|
+-------------+
|         11.0|
+-------------+



In [217]:
# mean
flights_df1.select(round(mean(col('DepDelay'))).alias('DepDelay mean')).collect()[0][0]

11.0

In [207]:
# mean upto 3 decimal points
flights_df1.select(round(mean(col('DepDelay')), 3).alias('DepDelay mean')).show()

+-------------+
|DepDelay mean|
+-------------+
|       10.619|
+-------------+



In [209]:
# variance
flights_df1.select(round(variance(col('DepDelay'))).alias('DepDelay Variance')).show()

+-------------+
|DepDelay mean|
+-------------+
|       1310.0|
+-------------+



In [214]:
# median - use approaxQuantile().expr()

flights_df1.approxQuantile('ArrDelay', [0.5], 0.0001)[0]

-3.0

#### 19. GroupBy & agg()

In [220]:
flights_df1.groupBy('Carrier').agg(mean("DepDelay").alias('mean DepDelay')).show()

+-------+------------------+
|Carrier|     mean DepDelay|
+-------+------------------+
|     UA|12.644186783024843|
|     AA|12.154096505870111|
|     EV| 14.52813602113455|
|     B6|12.675216069471794|
|     DL| 7.451940716867138|
|     OO| 7.954327121364983|
|     F9|12.142480802645592|
|     YV| 9.595018289496604|
|     US| 5.011879623272143|
|     MQ|15.612577198673192|
|     HA|1.5358414704192993|
|     AS|0.6606730403765751|
|     FL|10.227206113831024|
|     VX|14.416962353959326|
|     WN|12.930658050935614|
|     9E| 9.767838809034908|
+-------+------------------+



In [221]:
flights_df1.groupBy('Carrier').agg(mean("ArrDelay"), mean("DepDelay")).show()

+-------+--------------------+------------------+
|Carrier|       avg(ArrDelay)|     avg(DepDelay)|
+-------+--------------------+------------------+
|     UA|   5.207155029152466|12.644186783024843|
|     AA|    7.22786703097812|12.154096505870111|
|     EV|  10.501436641191532| 14.52813602113455|
|     B6|   9.679335778153199|12.675216069471794|
|     DL|  2.8085929091567747| 7.451940716867138|
|     OO|   6.447766785619039| 7.954327121364983|
|     F9|  12.870312237233028|12.142480802645592|
|     YV|   8.749505833107245| 9.595018289496604|
|     US|   3.957719324788726| 5.011879623272143|
|     MQ|   14.27679662028746|15.612577198673192|
|     HA|   1.534325271442523|1.5358414704192993|
|     AS|-0.27272328542814955|0.6606730403765751|
|     FL|   7.277437501357486|10.227206113831024|
|     VX|   9.678802215555043|14.416962353959326|
|     WN|   8.368672739670938|12.930658050935614|
|     9E|   4.931550031523011| 9.767838809034908|
+-------+--------------------+------------------+


#### 20. WithColumn
- all column operations.
- used to create new column or modify an existing column in a dataframe. 
- also used for applying transformations or calculations.

In [223]:
flights_df1.withColumn("TotalDelay", col("DepDelay") + col("ArrDelay")).show()

+----------+---------+-------+---------------+-------------+--------+--------+----------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|TotalDelay|
+----------+---------+-------+---------------+-------------+--------+--------+----------+
|         6|        1|     WN|          10821|        10140|       1|     -22|       -21|
|         8|        1|     AA|          11298|        10140|       0|       6|         6|
|        15|        1|     WN|          14747|        10140|      -6|       3|        -3|
|        27|        1|     AA|          11298|        10140|     113|     117|       230|
|         7|        2|     OO|          12266|        10140|      -3|     -11|       -14|
|        28|        2|     WN|          14107|        10140|      -3|       0|        -3|
|        30|        2|     OO|          12266|        10140|      -4|     -11|       -15|
|         1|        3|     EV|          12266|        10140|     -11|     -26|       -37|
|         

In [232]:
flights_df1.withColumn("isDelay", when((col('DepDelay') > 0) & (col('ArrDelay') > 0), "Delay").otherwise("No Delay")).show()

+----------+---------+-------+---------------+-------------+--------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay| isDelay|
+----------+---------+-------+---------------+-------------+--------+--------+--------+
|         6|        1|     WN|          10821|        10140|       1|     -22|No Delay|
|         8|        1|     AA|          11298|        10140|       0|       6|No Delay|
|        15|        1|     WN|          14747|        10140|      -6|       3|No Delay|
|        27|        1|     AA|          11298|        10140|     113|     117|   Delay|
|         7|        2|     OO|          12266|        10140|      -3|     -11|No Delay|
|        28|        2|     WN|          14107|        10140|      -3|       0|No Delay|
|        30|        2|     OO|          12266|        10140|      -4|     -11|No Delay|
|         1|        3|     EV|          12266|        10140|     -11|     -26|No Delay|
|         3|        3|     OO|  

#### 21. expr()


In [242]:
from pyspark.sql.functions import expr

median_expr = expr(f'percentile_approx({"ArrDelay"}, 0.5)')

median_value = flights_df.agg(median_expr.alias("Median")).collect()[0][0]